---
comments: true
layout: notebook
title: Final - Individual Review
type: hacks
courses: { compsci: {week: 12} }
---

## ScholarSearch Individual Review

### Project Overview
Our project centered around developing a program that would make the college application process easier. ScholarSearch allows you to search for colleges, pick favorites and sort them into a list, and includes features to help you pick the right college and create the best application. Those features include a quiz you can take to record your interests and recieve college recommendations, along with a selection of articles that pops up for you (from our database) with information on the college application process.  

### My Part: Article Database + Connection to Frontend

I worked most heavily on the article database that allowed articles to appear on the frontend for the user. My work focused on the backend and modeling my code after the login page to store articles (similar to storing user data) to be sent to the frontend for viewing.

#### articles.py - api Folder 

This is a Flask application that provides a RESTful API for managing articles. First I'm imporing the needed modules for the project. Then I have a Blueprint object (Article_bp) to organize anything with the name "article" in my code. "Article_api" is an Api object that the Bluprint is initialzing.

ArticleAPI and ArticleListAPI are subclasses to the Resource class, which is provided by Flask-RESTful to represent RESTful resources. ArticleAPI defines methods to handle GET, POST, and DELETE requests for individual articles.
ArticleListAPI defines a method to handle GET requests for a list of articles.

GET retrieves an article by its ID from the database and checks that it exists. If it does, it returns the article in JSON. Otherwise there will be a 404 error. POST parses the request arguments and creates articles to add to the database. DELETE parses the request arguments to get the ID of the article to delete;it'll find the article in the database to delete and will othewise report an error if the article isn't found.

GET for the Article List retrieves all articles from the database and returns a list of dictionaries. At the end Resource registration is done with specific endpoints ("/articles" and "/articlesList"). 
Resource Registration:

In [ ]:
from flask import Blueprint, request
from flask_restful import Api, Resource, reqparse
from .. import db
from ..model.articles import Article

Article_bp = Blueprint("Article", __name__, url_prefix='/articles')
Article_api = Api(Article_bp)


class ArticleAPI(Resource):
    def get(self):
        id = request.args.get("id")
        data = db.session.query(Article).get(id)
        if data:
            return data.to_dict()
        return {"message": "not found"}, 404

    def post(self): 
        parser = reqparse.RequestParser()
        parser.add_argument("title", type=str)
        parser.add_argument("author", type=str)
        parser.add_argument("link", type=str)
        args = parser.parse_args()
        data = Article(args["title"], args["author"], args["link"])

     try:
            db.session.add(data)  # Fix: Use `data` instead of `Article`
            db.session.commit()
            return data.to_dict(), 201
        except Exception as exception:
            db.session.rollback()
            return {"message": f"error {exception}"}, 500

    def delete(self):
        parser = reqparse.RequestParser()
        parser.add_argument("id", required=True, type=int)
        args = parser.parse_args()

        try:
            data = db.session.query(Article).get(args["id"])
            if data:
                db.session.delete(data)
                db.session.commit()
                return data.to_dict()
            else:
                return {"message": "not found"}, 404
        except Exception as exception:
            db.session.rollback()
            return {"message": f"error {exception}"}, 500

class ArticleListAPI(Resource):
    def get(self):
        articles = db.session.query(Article).all()
        return [article.to_dict() for article in articles]
        # Fix: Use `articles` instead of `article`


Article_api.add_resource(ArticleAPI, "/articles")
# Fix: Add a leading slash ("/articles")
Article_api.add_resource(ArticleListAPI, "/articlesList") 

#### articles.py - Model Folder

Here a SQLAlchemy model Article representing articles is being defined to create articles, turn them into dictionaries, and initilize the data. At the top is imports from SQLAlchemy.

Article is a SQLAlchemy model representing articles. It is subclassed from db.Model and defines columns for the id, title, author, and link. "__init__" initializes an Article instance with title, author, and link.
"to_dict" converts an Article instance to a dictionary. The create section is made to create new articles for the list, and will return an error if an exception happens in the process.

"init_articles" initializes sample article data, creating a list of dictionaries to the add the articles to the database. The main block then makes the necessary database tables based on the defined models. The data is then initialized, and we get some articles!

In [ ]:
from sqlalchemy import Column, Integer, String
from .. import db

class Article(db.Model):
    __tablename__ = 'articles'

    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(255), nullable=False)
    author = db.Column(db.String(255), nullable=False)
    link = db.Column(db.String(255), nullable=False)

    def __init__(self, title, author, link):
        self.title = title
        self.author = author
        self.link = link

    @staticmethod
    def create(title, author, link):
        try:
            article = Article(title=title, author=author, link=link)
            db.session.add(article)
            db.session.commit()
            return article
        except Exception as e:
            db.session.rollback()
            print(f"Error creating article: {e}")
            return None

    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "author": self.author,
            "link": self.link
        }

# Builds working data for testing


def init_articles():
    articles_data = [
        {'title': '12 Strategies to Writing the Perfect College Essay',
         'author': 'Pamela Reynolds',
         'link': 'https://shorturl.at/rszH7'},
        {'title': 'How to Write a College Essay: The Ultimate Guide',
         'author': 'College Essay Guy',
         'link': 'https://shorturl.at/doIQ2'},
        {'title': 'How to Write a Successful Common App Activites List',
         'author': 'College Essay Guy',
         'link': 'https://shorturl.at/asyLT'},
    ]
    for data in articles_data:
        Article.create(**data)


if __name__ == "__main__":
    # Create the database tables based on the defined models
    db.create_all()
    # Initialize articles
    init_articles()

#### Additions to main.py

Here I only added a few lines to create the Articles table along with the other ones that already existed. I added "from scholarSearchApi.api.articles import Article_bp" and "from scholarSearchApi.model.articles import init_articles" to import the article Blueprint and intilize it. I added "app.register_blueprint(Article_bp)" to actually register the Blueprint.

"init_articles()" was added under the initilization section, and the last block was unedited (it simply allows everything to run).

In [ ]:
from flask_cors import CORS
from scholarSearchApi import app, db

from scholarSearchApi.api.login import login_bp
from scholarSearchApi.api.data import data_bp
from scholarSearchApi.api.articles import Article_bp

from scholarSearchApi.model.login import init_login
from scholarSearchApi.model.data import init_data
from scholarSearchApi.model.articles import init_articles

app.register_blueprint(login_bp)
app.register_blueprint(data_bp)
app.register_blueprint(Article_bp)


@app.before_first_request
def init_db():
    with app.app_context():
        db.create_all()
        init_login()
        init_data()
        init_articles()


if __name__ == "__main__":
    cors = CORS(app)
    app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///./volumes/sqlite.db"
    app.run(debug=True, host="0.0.0.0", port="8199")

#### Article Database

This is the article database I generated using the above code.

![]({{ site.baseurl }}/images/articledatabase.png "Article Database")